Возьмите датасет housing.csv
Подготовьте данные для обучения.
Импортируйте подходящий для решения задачи класс градиентного бустинга
Сделайте оценку бустинга с помощью функции cross_val_score и средней квадратичной ошибки на обучающих данных.
Понизьте размерность исходных данных с помощью класса PCA и объяснением дисперсии не менее 95%.
Сколько главных компонент вы нашли?
Сделайте оценку бустинга с помощью функции cross_val_score и средней квадратичной ошибки на обучающих данных пониженной размерности.
Сравните полученные оценки. Сделайте вывод.


In [167]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, \
              auc, mean_squared_error, accuracy_score
import math
plt.rcParams["figure.figsize"] = (12,6)
from sklearn.model_selection import cross_val_score

In [153]:
df = pd.read_csv('housing.csv')

In [154]:
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [173]:
df.shape

(20640, 14)

In [156]:
# Заменим нулевые значения средним значением для столбика total_bedrooms
mediantb = df['total_bedrooms'].median()
df['total_bedrooms'].fillna(mediantb, inplace = True)

In [157]:
df=pd.get_dummies(df)

In [158]:
# Отделение меток: нам теперь надо отделить нашу целевую переменную от всех остальных.
#мы разделяем и формируем новый dataset with library pandas
X = df.drop(['median_house_value'], axis = 1)
#X = df.drop(['median_house_value','longitude','latitude'], axis = 1)
Y = df['median_house_value']

In [159]:
# Нужно создать данные для обучения и данные для проверки
from sklearn.model_selection import train_test_split
# импортируем функцию train_test_split и передаем массив независимых переменных : X и массив меток: Y 
# и она по одинаковым индексам разбивает каждый массив на 2:на обучение (X_train,Y_train) и тестирование (X_test,Y_test ),
# Создаем соотнеште в котором нам надо побить все (test_size = 0.2)- в данном случает в тестовое множество попадет 20% всего количества
#  random_state инициализирует генерацию случайных чисел

X_train, X_test, Y_train,  Y_test =  train_test_split(X, Y, test_size = 0.2,random_state = 1)

# GradientBoostingRegressor

In [160]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, cross_val_predict


reg = GradientBoostingRegressor(random_state=1)
reg.fit(X_train, Y_train)

GradientBoostingRegressor(random_state=1)

In [161]:
pred_train = reg.predict(X_test)

In [162]:
squared_error = mean_squared_error(Y_test,pred_train)

In [163]:
squared_error_result = np.sqrt(squared_error)

In [164]:
print('средняя квадратичная ошибка на тестовом множестве:',squared_error_result)

средняя квадратичная ошибка на тестовом множестве: 54694.700975065585


cross_val_score

In [165]:
#передаем наш классификатор обученный, и наше множество,на котором учим.cv - то сколько разбиений мы хотим делать
score = cross_val_score(reg, X_train, Y_train, cv=5,scoring ='neg_mean_squared_error')
print('Результаты score:',score)

Результаты score: [0.76310969 0.78539891 0.76161178 0.78021511 0.75867357]


In [166]:
print('среднее значение по 5-ти результатам score:',score.mean())

среднее значение по 5-ти результатам score: 0.7698018131404525


# Снижение размерности

In [187]:
# Понизим размерность с //// до 8
from sklearn.decomposition import PCA
# pca = PCA(n_components=...) - создаем наш преобразователь
pca = PCA(n_components=0.95)
# X_centered - центрирование наших данных
X_centered = X - X.mean(axis=0)
# pca.fit(X_centered) - обучили с помощью fit наш трансформер(преобразователь )на основе центрированных данных
pca.fit(X_centered)
# получаем наши преобразованные данные, фактически проекцияна координаты с самыми хорошими дисперсиями!
X_pca = pca.transform(X_centered)

In [190]:
X_pca.shape

(20640, 2)

In [182]:
X_train, X_test, Y_train,  Y_test =  train_test_split(X_pca, Y, test_size = 0.2, random_state =1)
reg = GradientBoostingRegressor(random_state=1)
reg.fit(X_train, Y_train)
pred_train = reg.predict(X_test)
squared_error = mean_squared_error(Y_test,pred_train)
squared_error_result = np.sqrt(squared_error)
print('средняя квадратичная ошибка на тестовом множестве:',squared_error_result)

средняя квадратичная ошибка на тестовом множестве: 100831.937361348


In [183]:
score = cross_val_score(reg, X_train, Y_train, cv=5 ,scoring ='neg_mean_squared_error')
print('Результаты score:',score)

Результаты score: [0.2262423  0.26814387 0.22226633 0.25465965 0.24133915]


In [184]:
print('среднее значение по 5-ти результатам score:',score.mean())

среднее значение по 5-ти результатам score: 0.24253025848270954


In [185]:
#этот атрибут, объяснение:у нас есть два измерения и по каждому измерению он считает,
#какое общее количество дисперсии объяснено каждым вектором
# ответ: array([0.94996405, 0.04471501])
# в нашем случае первый собственный вектор объясняет 95% дисперсии
#
pca.explained_variance_ratio_

array([0.94996405, 0.04471501])

Сколько главных компонент вы нашли?
 n_components=0.95

Результаты после применения снижения размерности стали намного хуже. Можно сделать вывод, что бустингу снижение размерности противопоказано.